In [2]:
from __future__ import print_function
%matplotlib inline
import matplotlib.pylab as plt
import sys, os, glob
import numpy as np
import subprocess
import datetime
import time

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

from IPython.display import HTML
import xml.etree.ElementTree as ET
try:
    tree = ET.parse(os.environ['HADOOP_CONF_DIR'] + '/yarn-site.xml')
except IOError:
    raise IOError("Can't find the yarn configuration -- is HADOOP_CONF_DIR set?")

In [3]:
root = tree.getroot()
yarn_web_app = root.findall("./property[name='yarn.resourcemanager.webapp.address']")[0].find('value').text
yarn_web_app_string = "If this works successfully, you can check the <a target='_blank' href='http://{yarn_web_app}'>YARN application scheduler</a> and you should see your app listed there. Clicking on the 'Application Master' link will bring up the familiar Spark Web UI. "

Load spark configuration setup when executing notebook by https://github.com/rokroskar/spark_workshop

In [4]:
# configuration directory
os.environ['SPARK_CONF_DIR'] = os.path.realpath('../../spark_workshop/notebooks/twitter_dataframes/spark_config')

# number of cores
ncores = int(os.environ.get('LSB_DJOB_NUMPROC', 1)) 

# here we set the memory we want spark to use for the driver JVM
os.environ['SPARK_DRIVER_MEMORY'] = '%dG'%(ncores*0.7)

# python executable
os.environ['PYSPARK_PYTHON'] = subprocess.check_output('which python', shell=True).rstrip()

In [5]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, HiveContext
import pyspark.sql.functions as func
from pyspark.sql import Row, Window
from pyspark.sql.types import IntegerType, ArrayType, StringType, NullType, LongType, StructField, \
                              StructType, DateType, DataType, DateConverter, DatetimeConverter, \
                              TimestampType, BooleanType

### Start context

In [6]:
conf = SparkConf()
sc = SparkContext(master='yarn-client', conf=conf)

### Load twitter data

In [7]:
hc = HiveContext(sc)

Read in the parquet files by Iza and Rok containing the twitter data

In [8]:
data = hc.read.parquet('/user/roskarr/twitter/2014_12*')

### Read in our list containing the keywords

In [9]:
def get_keys(path='../docs/keywords.txt'):
    keys = np.genfromtxt(path, dtype="|S20", delimiter='#', autostrip=True)
    return keys.tolist()

In [10]:
keys = get_keys()
print(keys)
keys = keys[80:]

['isis', 'terrorism', 'arab', 'spring', 'attack', 'god', 'christian', 'allah', 'islam', 'syria', 'refugees', 'migrants', 'africa', 'italy', 'ethiopia', 'asylum', 'unhcr', 'immigration', 'foreigners', 'crowded', 'ebola', 'guinea', 'sierra', 'leone', 'liberia', 'virus', 'epidemic', 'vaccine', 'who', 'influenza', 'flu', 'birds', 'swine', 'pig', 'bitcoin', 'rosetta', 'comet', 'higgs', 'climate', 'doomsday', 'maya', 'curiosity', 'sandy', 'hurricane', 'black', 'white', 'mandela', 'nelson', 'left', 'right', 'mh17', 'mh370', 'ukraine', 'crimea', 'russia', 'snowden', 'nsa', 'obama', 'putin', 'pope', 'unemployment', 'boston', 'marathon', 'london', 'europe', 'usa', 'philippines', 'sochi', 'olympics', 'geneva', 'apple', 'linux', 'PC', 'google', 'iphone', 'galaxy', 'watch', 'facebook', 'twitter', 'whatsapp', 'vegan', 'gluten', 'vegetarian', 'meat', 'pasta', 'banana', 'family', 'divorce', 'marriage', 'wedding', 'holidays', 'homework', 'television', 'coffee', 'tea', 'school', 'work', 'teacher', 'spor

### Custom functions performing operations on the columns

In [11]:
# Change the date format to YYYY-MM-DD HH:MM:SS
convert_date_string = func.udf(lambda date_string: datetime.date.strftime(datetime.datetime.strptime(date_string, \
                                                                           '%a %b %d %H:%M:%S +0000 %Y'),\
                                                                           '%Y-%m-%d %H:%M:%S'
                                                                         ), StringType())

# Convert the date string to a datetime object
datetime_udf = func.udf(lambda date_string: datetime.strptime(date_string, '%a %b %d %H:%M:%S +0000 %Y'), DateType())

# Map the datetime object column to a unique number for every day in any year
new_date = (lambda col: func.dayofyear(col) + func.year(col)*1000)

# Convert the tweet text string to lowercase and split
text_udf = func.udf(lambda row: row.lower().split(), returnType=ArrayType(StringType()))

# Boolean containing whether a keyword appears in the text
keys_filter_udf = func.udf(lambda row: len([val for val in row if val in keys]) > 0, returnType=BooleanType())

# Returns list of keys contained in text
keys_list_udf = func.udf(lambda row: [val for val in row if val in keys], returnType=ArrayType(StringType()))

### Filter dataframe

Number of partitions

In [12]:
Npartitions = sc.defaultParallelism*200
print(Npartitions)

16000


In [13]:
keywords_df = (data.select('created_at', 'text')
                   .filter(func.length('text') > 0)
                   .withColumn('created_at', convert_date_string('created_at'))
                   .withColumn('created_at', new_date('created_at'))
                   .withColumn('text', text_udf('text'))
                   .repartition(Npartitions)
                   .filter(keys_filter_udf('text'))
                   .repartition(Npartitions)
                   .withColumn('keys', keys_list_udf('text'))
                   .select('created_at', 'keys')
               )
keywords_df.cache()

DataFrame[created_at: int, keys: array<string>]

In [14]:
outdir = '/cluster/home/phys/bruclaud/Documents/spuriousrelations/data/'

In [ ]:
import gc

In [ ]:
for key in keys:
    collected = gc.collect()
    print('Garbage collected %d objects' % collected)
    start = time.time()
    csvlist = (keywords_df.repartition(Npartitions)
                          .filter(func.udf(lambda row: key in row, returnType=BooleanType())('keys'))
                          .repartition(Npartitions)
                          .groupBy('created_at')
                          .count()
                          .rdd
                          .map(lambda x: ",".join(map(str, x)))
                          .collect())
    np.savetxt(outdir+key+'.csv', np.array(csvlist), fmt='%s')
    print(key, ': Time: {:.1f}s'.format(time.time()-start))

In [ ]:
sc.stop()